### Setup

In [1]:
import numpy as np

from hot_dust import preprocess, model

In [2]:
from importlib import reload

preprocess = reload(preprocess)
model = reload(model)

In [3]:
ds = preprocess.prepare_training_data()
train, validate, test = preprocess.to_tensorflow(preprocess.split_training_data(ds))

### Parameters

In [4]:
epochs = 500 # Max 500
batch_size = 64
buffer_size = 10 * batch_size

model = reload(model)  # DEBUG

train = train.shuffle(buffer_size).batch(batch_size)
validate = validate.batch(batch_size)

layer = model.pretraining(dataset=test)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [5]:
network = model.compile(normalization=layer)
network.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 11)]              0         
                                                                 
 normalization (Normalizatio  (None, 11)               23        
 n)                                                              
                                                                 
 dense (Dense)               (None, 16)                192       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 360
Trainable params: 337
Non-trainable params: 23
______________________________________________________________

### Fitting

In [6]:
fit = network.fit(
    train,
    epochs=epochs,
    validation_data=validate,
    verbose=2,
) # TODO add early stopping callback
network.save("data/network")
fit = {"epoch": fit.epoch, **fit.history}
np.savez("data/fit.npz", fit)

Epoch 1/500
3122/3122 - 6s - loss: 0.6929 - val_loss: 0.4403 - 6s/epoch - 2ms/step
Epoch 2/500
3122/3122 - 4s - loss: 0.3793 - val_loss: 0.3325 - 4s/epoch - 1ms/step
Epoch 3/500
3122/3122 - 7s - loss: 0.3059 - val_loss: 0.2809 - 7s/epoch - 2ms/step
Epoch 4/500
3122/3122 - 6s - loss: 0.2640 - val_loss: 0.2472 - 6s/epoch - 2ms/step
Epoch 5/500
3122/3122 - 5s - loss: 0.2344 - val_loss: 0.2219 - 5s/epoch - 2ms/step
Epoch 6/500
3122/3122 - 5s - loss: 0.2108 - val_loss: 0.2016 - 5s/epoch - 2ms/step
Epoch 7/500
3122/3122 - 5s - loss: 0.1934 - val_loss: 0.1868 - 5s/epoch - 1ms/step
Epoch 8/500
3122/3122 - 4s - loss: 0.1808 - val_loss: 0.1758 - 4s/epoch - 1ms/step
Epoch 9/500
3122/3122 - 6s - loss: 0.1697 - val_loss: 0.1644 - 6s/epoch - 2ms/step
Epoch 10/500
3122/3122 - 4s - loss: 0.1597 - val_loss: 0.1554 - 4s/epoch - 1ms/step
Epoch 11/500
3122/3122 - 5s - loss: 0.1512 - val_loss: 0.1474 - 5s/epoch - 2ms/step
Epoch 12/500
3122/3122 - 6s - loss: 0.1444 - val_loss: 0.1409 - 6s/epoch - 2ms/step
E

INFO:tensorflow:Assets written to: data/network\assets


INFO:tensorflow:Assets written to: data/network\assets


### Training and Validation Losses

In [7]:
model.plot_loss(fit)

:NdOverlay   [Variable]
   :Scatter   [epoch]   (value)